In [37]:
## import auto reload
%load_ext autoreload
%autoreload 2

from simulation import run_simulation, initialize_simulation, MetricsHandler, InteractionHandler

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
num_employees = 100
num_teams = 10
min_team_size = 8
num_friendships = 5
num_spaces = 20
space_types = ["Workstation"] * 10 + ["Meeting Room"] * 5 + ["Break Area"] * 3 + ["Quiet Area"] * 2
space_capacities = [8] * 10 + [15] * 5 + [20] * 3 + [10] * 2
hierarchy_levels = 5
hierarchy_adjustment_factor = 0.1

emps, spaces, team_graph, friendship_graph = initialize_simulation(num_employees, num_teams, min_team_size,
                                                                   num_friendships, num_spaces, space_types, space_capacities, hierarchy_levels, hierarchy_adjustment_factor)

# run_simulation(emps, spaces, 10, initial_info_holder=6, team_graph=team_graph, friendship_graph=friendship_graph)


In [ ]:
def run_simulation(emps, spaces, time_steps, initial_info_holder, team_graph, friendship_graph):
    emps[initial_info_holder - 1].info.add("Important Information") ## add initial information

    ## initialize handlers
    interaction_handler = InteractionHandler(team_graph, friendship_graph)
    metrics_handler = MetricsHandler(emps, spaces)

    ## add metricshandler to employees
    for emp in emps:
        emp.metrics_handler = metrics_handler

    ## run simulation
    for t in range(time_steps):
        # Update employee locations
        for emp in emps:
            cur_space = emp.loc
            if cur_space is not None:
                cur_space.remove_occupant(emp)

            emp.move(spaces, threshold=3, timestep=t)
            new_space = emp.loc
            new_space.add_occupant(emp)

        # Simulate interactions between employees in the same space
        interaction_handler.handle_interactions(spaces, timestep=t)

        # Calculate and print metrics
        informed_count, informed_percentage = metrics_handler.calculate_metrics()
        metrics_handler.print_simulation_state(t, informed_count, informed_percentage)